### Import libraries

In [19]:
# beautidulsoup as scrapper
from bs4 import BeautifulSoup

# selenium for automate scrapping process
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# other libraries needed
import time
import pandas as pd

### Webdriver instantiate

In [20]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
options = Options()
# options.add_argument('--headless=new') # uncomment this when require to run in headless mode

# launch the driver
url = "https://www.vlr.gg/events" # directly go to event tab (all results are stored in there)
driver.get(url)
driver.maximize_window()
main_tab_handle = driver.current_window_handle # to locate first tab
driver.implicitly_wait(5) # wait for the content to load completely before scrape (5 sec)

### Interact with web content/ Scrapping

In [21]:
# scrape function
def scrape(a):
    # open new tab to show event detail
    action = ActionChains(driver)
    action.key_down(Keys.CONTROL).click(a).key_up(Keys.CONTROL).perform()
    driver.switch_to.window(driver.window_handles[1])
    event_window = driver.current_window_handle
    
    # navigate to matches tab
    nav_bar = driver.find_element(By.CLASS_NAME, 'wf-nav')
    matches = nav_bar.find_elements(By.TAG_NAME, 'a') 
    matches[1].click() # all matches record are stored in matches tab [second tab]
    matches_day = driver.find_elements(By.CLASS_NAME, 'wf-card')
    
    for day in matches_day[1:]: # first element with class='wf-card' is the event header, skip it
        all_matches = day.find_elements(By.TAG_NAME, 'a') # find all clickable element in the div
        
        for match in all_matches: # find all matches & scrape
            # open match details in new tab
            action.key_down(Keys.CONTROL).click(match).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[2])
            time.sleep(1)
            
            # scrape: TODO
            
            driver.close()# return to previous page and continue next loop
            driver.switch_to.window(event_window)
            
            
    driver.close() # close event window
    time.sleep(3)
    driver.switch_to.window(main_tab_handle) # focus to event list

    

In [22]:
# event-container --> .wf-label .mod-large .mod-completed text == "completed events" to locate completed event div
events_div = driver.find_elements(By.CLASS_NAME, 'events-container-col')
completed_events_div = events_div[1] # completed events always == second div

# extract all clickable element into an array
all_a = completed_events_div.find_elements(By.TAG_NAME, 'a')

# scrape all completed event
for a in all_a[:2]:
    scrape(a)


### Exit driver, complete scrapping

In [23]:
# quit driver after get the html
# driver.quit()